In [1]:
from os.path import join
import pandas as pd
import xlsxwriter
import openpyxl
import os
import numpy as np
import itertools
import itertools as it
from pandas import Series, ExcelWriter
import csv
from skbio.stats.composition import clr
import plotly.express as px
import matplotlib.pyplot as plt

#### Variables

In [2]:
top_path = '../SPECIFIC_PAPERS/'

db_path1 = top_path + 'Ravel_et_al.2013/Hiseq_sequencing/DADA2_results/'
db_path2 = top_path + 'Ravel_et_al.2013/454_Sequencing/DADA2_results/DADA_output_files/'
db_path3 = top_path + 'Carter_et_al.2022/DADA2_results/'
db_path4 = top_path + 'Ravel_et_al.2011/DADA2_results/'
db_path5 = top_path + 'Serrano_et_al.2019/noDuplicatedID/DADA2_results/'
db_path6 = top_path + 'Srinivasan_et_al.2012/DADA2_results/'

asv_abun_file_name = 'abundance_ASV.csv'
taxa_file_name = 'taxa_ASV.csv'

speciate_filename1 = 'Ravel2013_Hiseq_2step_single_V3V4_MC_order7.txt'
speciate_filename2 = 'Ravel2013_V1V3_454_all_MC_order7_rev.txt'
speciate_filename3 = 'Carter2022_V3V4_MC_order7.txt'
speciate_filename4 = 'Ravel2011_V1V3_MC_order7.txt'
speciate_filename5 = 'Serrano2019_MC_order7.txt'
speciate_filename6 = 'Srinivasan2012_V3V4_MC_order7.txt'

taxa_names_lst = ['k','p','c','o','f','g','s']
names_lst = ['Lactobacillus', 'Gardnerella', 'Prevotella', 'Atopobium', 'Shuttleworthia', 'Sneathia', 'BVAB', 'Unknown']
date = '16052023'

#### Functions

In [3]:
def order_tax(df):
    new_df = df.copy()
    new_df = df.fillna(0)
    new_df.columns = taxa_names_lst
    new_df['cond_spec'] = np.nan
    
    return new_df

In [4]:
def get_cond(row):
    if type(row['s']) == str:
        row['cond_spec'] = row['g'] + '_' + row['s']
    
    else:
        for i in range(len(taxa_names_lst)):
            curr_level = taxa_names_lst[i]
            if row[curr_level] == 0:
                if curr_level == 'k':
                    row['cond_spec'] = np.nan
                else:
                    upper_level = taxa_names_lst[i - 1]
                    row['cond_spec'] = upper_level + '_' + row[upper_level]
                break
    
    return row

In [5]:
def order_speciate(df):
    df.columns = ['ASV', 'speciate_spec', 'posterior_probability', 'decisions_num']
    df.drop(['posterior_probability', 'decisions_num'], axis = 1, inplace = True)
    df.set_index('ASV', inplace = True)
    
    return df

In [6]:
def add_final_name(row):
    if any(name in row['speciate_spec'] for name in names_lst) or 'BVAB' in row['speciate_spec']:
        return row['speciate_spec']
    else:
        if row['g'] is not 'Unknown' and '00' not in row['g']:
            return 'g_' + row['g']
        elif row['f'] != 0:
            return 'f_' + row['f']
        elif row['s'] == 0: 
            return row['cond_spec']
        else:
            return 'Unknown'

In [7]:
def get_new_name(name):
    if name.split('_')[0] == 'f':
        return name
    elif 'Cluster' in name:
        return name
    elif len(name.split('_')[0]) == 1 and name.split('_')[0] != 'f':
        return '_'.join(name.split('_')[:2])
    elif any(map(str.isdigit, name.split('_')[1])):
        return '_'.join([name.split('_')[i] for i in [0, 2]])
    else: 
        return '_'.join(name.split('_')[-2:])

In [19]:
def get_spec_df(path, spec_path):
    ## DADA tax df
    tax_df_orig = pd.read_csv(path + taxa_file_name, index_col = 0)
    print('Tax_df row num: ' + str(tax_df_orig.shape[0]))
    
    tax_df = order_tax(tax_df_orig)
    tax_df = tax_df.apply(lambda row: get_cond(row), axis = 1)
    tax_df['g'] = tax_df['g'].replace({0: 'Unknown'})
    
    ## Speciate df
    speciate_df = pd.read_csv(path + spec_path, sep = '\t', header = None)
    speciate_df = order_speciate(speciate_df)
    
    ## Merge both dfs
    merged_tax_df = tax_df.join(speciate_df)
    
    ## Create final name column
    merged_tax_df['final_name'] = merged_tax_df.apply(add_final_name, axis = 1)
    
    ## Replace long names
    long_names_lst = [name for name in merged_tax_df.final_name.unique() if type(name) == str and name.count('_') > 1]
    
    new_names = []
    for name in long_names_lst:
        new_name = get_new_name(name)
        new_names.append(new_name)
        
    replacers = dict(zip(long_names_lst, new_names))
    replacers.update({'Lactobacillus_ultunensis': 'Lactobacillus_crispatus'})

    merged_tax_df['final_name'] = merged_tax_df['final_name'].replace(replacers)
    merged_tax_df = merged_tax_df.loc[(merged_tax_df['p'] != 0)]
    print('Merged tax df after removal of k_ ASVs: ' + str(merged_tax_df.shape[0]))
    
    long_names_lst = [name for name in merged_tax_df.final_name.unique() if name.count('_') > 1]
    
    return merged_tax_df   

In [9]:
def get_spec_reads_table(path, merged_tax_df):
    abun_df = pd.read_csv(path + asv_abun_file_name, index_col = 0)
    print('Orig reads table columns: ' + str(abun_df.shape[1]))
    
    filt_abun_df = abun_df[merged_tax_df.index]
    abun_df_t = filt_abun_df.T
    spec_abun_df = pd.merge(merged_tax_df[['final_name']], abun_df_t, left_index = True, right_index = True)

    spec_abun_df.reset_index(drop = True, inplace = True)
    spec_abun_df.set_index('final_name', inplace = True)
    spec_abun_df.index.name = None
    spec_abun_df = spec_abun_df.T
    print('Final reads table columns: ' + str(spec_abun_df.shape[1]))
        
    return spec_abun_df

In [10]:
def get_filtered(df):
    filt_df = df.loc[df.sum(axis = 1) >= 1000]
    filt_df02 = filt_df.loc[:, filt_df.sum(axis = 0) >= 100]
    filt_df03 = filt_df02.drop(filt_df02.loc[filt_df02.sum(axis = 1) == 0].index, inplace = False)
    
    print('Shape after filter- rows: ' + str(filt_df03.shape[0]) + ', columns: ' + str(filt_df03.shape[1]))

    return filt_df03

In [11]:
def sum_spec(df):
    sum_spec_df = df.groupby(df.columns, axis = 1).sum()
    print('Sum of species columns: ' + str(sum_spec_df.shape[1]))
    
    return sum_spec_df

In [12]:
def get_valencia_input(df):   
    valencia_df = df.copy()
    valencia_df.reset_index(inplace = True)
    valencia_df = valencia_df.rename(columns = {'index':'sampleID'})
    read_count_values = valencia_df.sum(axis = 1)
    valencia_df.insert(loc = 1, column = 'read_count', value = read_count_values)
    
    return valencia_df

In [13]:
def save_and_cmd_valencia(path, df):
    valencia_file_name = 'valencia_df_' + date 
    df.to_csv(path + valencia_file_name + '.csv', index = False)
    
    in_file = path + valencia_file_name + '.csv'
    out_file = path + valencia_file_name + '_res'
    cmd = 'py C:/Users/morts/Documents/TAU/Python_Scripts/Manifold_Project/Valencia/Valencia.py -ref C:/Users/morts/Documents/TAU/Taxonomy/CST_centroids_19042023.csv -i ' + in_file + ' -o ' + out_file
    print('*** cmd for VALENCIA ***')
    print(cmd)
    
    return 

In [17]:
def all_func(name, path, spec_path):
    ## Table with ASV and taxa final
    merged_tax_df = get_spec_df(path, spec_path)
    
    ## Reads table with taxa names (after sum)
    spec_abun_df = get_spec_reads_table(path, merged_tax_df)
    
    ## Filtering
    filt_df = get_filtered(spec_abun_df)
    sum_spec_df = sum_spec(filt_df)
    
    ## VALENCIA input
    input_valencia = get_valencia_input(sum_spec_df)
#     save_and_cmd_valencia(path, input_valencia)
    
    print('****  ****  ****')
    
    return merged_tax_df

#### Run

In [16]:
db_lst = [db_path1, db_path2, db_path3, db_path4, db_path5, db_path6]
speciate_lst = [speciate_filename1, speciate_filename2, speciate_filename3, speciate_filename4, speciate_filename5, speciate_filename6]

In [20]:
for i, path in enumerate(db_lst):
    df = all_func('test', db_lst[i], speciate_lst[i])
    print('***')

Tax_df row num: 22633
Merged tax df after removal of k_ ASVs: 17495
Orig reads table columns: 22633
Final reads table columns: 17495
Shape after filter- rows: 3412, columns: 8638
Sum of species columns: 278
****  ****  ****
***
Tax_df row num: 6676
Merged tax df after removal of k_ ASVs: 6545
Orig reads table columns: 6676
Final reads table columns: 6545
Shape after filter- rows: 1573, columns: 1516
Sum of species columns: 82
****  ****  ****
***
Tax_df row num: 3598
Merged tax df after removal of k_ ASVs: 3544
Orig reads table columns: 3598
Final reads table columns: 3544
Shape after filter- rows: 706, columns: 1576
Sum of species columns: 137
****  ****  ****
***
Tax_df row num: 4851
Merged tax df after removal of k_ ASVs: 4840
Orig reads table columns: 4851
Final reads table columns: 4840
Shape after filter- rows: 385, columns: 875
Sum of species columns: 54
****  ****  ****
***
Tax_df row num: 37719
Merged tax df after removal of k_ ASVs: 37240
Orig reads table columns: 37719
Final